In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
compustat = pd.read_csv('/content/drive/MyDrive/filtered_compustat_char.csv')
crsp = pd.read_csv("/content/drive/MyDrive/final_crsp.csv")

In [ ]:
target = crsp[crsp['date'].between('2004-01-01', '2024-12-31')]
compustat = compustat[compustat["datadate"].between('2004-01-01', '2024-12-31')]

In [ ]:
target.reset_index(drop=True, inplace=True)
compustat.reset_index(drop=True, inplace=True)

In [ ]:
for i in target.columns:
  print(i, target[i].isna().sum())

PERMNO 0
HdrCUSIP 0
CUSIP 12288
Ticker 32207
TradingSymbol 32207
PERMCO 0
SICCD 0
NAICS 0
MthCalDt 0
MthRet 20567
sprtrn 0
date 0
mom_12 95880
mom_6 51630
vol_12 87733
vol_6 47511
rev_1 20567
id 0
rvol_1 21298
qmj_safety 0
seas_11_15na 0
ret_3_1 0
iskew_ff3_21d 0
rskew_21d 0
sti_gr1a 0
earnings_variability 0
nfna_gr1a 0
seas_16_20an 0
corr_1260d 0
resff3_12_1 0


In [ ]:
# Truncate each CUSIP to its first 8 characters
compustat['cusip'] = compustat['cusip'].str[:8]

target.drop(columns=['CUSIP'], inplace=True)
target.rename(columns={'HdrCUSIP': 'cusip'}, inplace=True)

compustat['datadate'] = pd.to_datetime(compustat['datadate'])
compustat.rename(columns={'datadate': 'date'}, inplace=True)
target['date'] = pd.to_datetime(target['date'])

In [ ]:
target = target.sort_values('date')
compustat = compustat.sort_values('date')
target["quarter"] = target["date"].dt.to_period('Q')
compustat["quarter"] = compustat["date"].dt.to_period('Q')

target['cusip'] = target['cusip'].astype(str)
compustat['cusip'] = compustat['cusip'].astype(str)
compustat.drop(columns = ["date"],inplace=True)
# We use 'cusip' here as per your request, but 'gvkey' is often recommended.
merged_df = target.merge(compustat, on = ["cusip","quarter"],how="left")

In [ ]:
merged_df.dropna(inplace = True)

In [ ]:
merged_df.drop_duplicates(inplace=True)

In [ ]:
merged_df = merged_df.reset_index(drop=True)

In [ ]:
merged_df

,PERMNO,cusip,Ticker,TradingSymbol,PERMCO,SICCD,NAICS,MthCalDt,MthRet,sprtrn,...,conm,capxy,chechy,cshfdy,cshpry,dltry,dpcy,epspxy,oibdpy,txty
0,10001,36720410,EWST,EWST,7953,4920,0,2004-01-30,0.010084,0.017276,...,GAS NATURAL INC,1.405000,2.461000,2.596000,2.596000,0.000000,2.178000,0.110000,3.678000,0.151000
1,10853,76017410,RENT,RENT,8738,7822,0,2004-01-30,0.011294,0.017276,...,RENTRAK CORP,1.597000,-1.328000,10.119000,9.600000,0.068000,0.875000,0.150000,3.210000,0.479000
2,18761,79547710,SBF,SBF,21093,6726,0,2004-01-30,0.033250,0.017276,...,SALOMON BROTHERS FUND INC,2.575966,40.350690,44.996000,44.119379,0.000000,8.073828,0.686552,58.224724,9.804276
3,85729,84917U10,TSA,TSA,15920,5941,0,2004-01-30,0.100260,0.017276,...,SPORTS AUTHORITY INC,79.472000,17.853000,19.480000,18.309000,700.406000,40.710000,0.890000,117.273000,7.653000
4,80658,70532310,PSAI,PSAI,13126,8080,0,2004-01-30,0.186722,0.017276,...,PEDIATRIC SVCS AMERICA INC,2.788000,-3.880000,7.340000,6.901000,0.059000,1.843000,0.460000,7.984000,1.995000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1655277,19425,31609234,FBCV,FBCV,54581,6726,525990,2024-12-31,-0.068526,-0.024990,...,FIDELITY BLUE CHIP VALUE ETF,10.459333,12.598000,62.096310,59.173859,47.844889,18.977556,0.049014,32.005111,1.560352
1655278,19426,31609235,FBCG,FBCG,54581,6726,525990,2024-12-31,0.006564,-0.024990,...,FIDELITY BLUE CHIP GROWT ETF,39.272818,-3.378045,63.615902,62.955902,70.011932,40.354273,0.294773,104.843432,14.175955
1655279,19427,31609236,FFLC,FFLC,54581,6726,525990,2024-12-31,-0.033064,-0.024990,...,FIDELITY FUND LAR CAP CO ETF,8.328222,11.878000,62.081662,59.439305,43.502204,16.164315,0.047136,27.367796,1.580070
1655280,19405,98980F10,ZI,ZI,57093,9999,513210,2024-12-31,-0.039305,-0.024990,...,ZOOMINFO TECHNO INC,64.900000,-307.200000,362.189000,361.953000,5.900000,155.500000,0.080000,314.400000,2.200000


In [ ]:
merged_df.to_csv("/content/drive/MyDrive/new_data.csv")